Purpose of this notebook is to get full text articles from PMC

In [17]:
# imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os, json
import requests
import time

import sys
sys.path.append('../pubmed_rag')

from bioc import get_biocjson, passages_to_df

# import nltk
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')

#from spellchecker import SpellChecker

In [9]:
df = pd.read_csv('data/test_pmid_list.csv', header=None)
df.head()

pmids = df[0].astype(str).to_list()

### Current status 

Getting full texts via biocjson from [Pubtator](https://www.ncbi.nlm.nih.gov/research/pubtator3/api) instead of [BioC API for PMC](https://www.ncbi.nlm.nih.gov/research/bionlp/APIs/BioC-PMC/)

Why?? 
- I only have pmids currently, and Pubtator automatically links pmid to pmcid to return full text if it is available, otherwise returns abstract.
- BioC API for PMC is not compatible with pmid id even though it says it does

Bioc files
- it looks like each 'passage' is a sentence
- and the passages have annotations 
- additionally, relevant to us, the passage has metadata such as what section its from


Steps
- config
    - provide list of pmids 
    - output path
- retrieve biocjson (full text or abstract) and save to output path
- for each json parse for text + section

Vertex database of sentences or sections?
Option 1:
- embedd each sentence using [PubMedBERT now called BiomedBERT](https://huggingface.co/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext)
- keeping metadata

Option 2:
- collapse the sentences into one section 
- embed each section using same model




In [18]:
# Example usage
sample = pmids[:15]


In [19]:
for each in sample:
    
    result = get_biocjson(each, 'biocjson')

    if result is not None:
        passages_to_df(result, 'biocjson')

    time.sleep(2)